In [55]:
# Imports
import os
import asyncio
from typing import cast
from dotenv import load_dotenv
from typing import Dict
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent


# Arize Phoenix Tracer
from phoenix.otel import register
from opentelemetry import trace as otel_trace


# Markdown output display
from IPython.display import Markdown, display

In [56]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [57]:
# Load Environment Variables
load_dotenv(override=True)

# Ollama Environment Variables
ollama_api_key = env_to_str('OLLAMA_API_KEY')
ollama_base_url = env_to_str('OLLAMA_BASE_URL')
model_llama = env_to_str('MODEL_LLAMA3')

# Phoenix Collector Variables
phoenix_collector_endpoint = env_to_str("PHOENIX_COLLECTOR_ENDPOINT") or "http://localhost:6006/v1/traces"

# Maileroo Environment Variables
maileroo_api_key = env_to_str("MAILEROO_API_KEY")
maileroo_base_url = env_to_str("MAILEROO_BASE_URL")
maileroo_template_url = env_to_str("MAILEROO_TEMPLATE_URL")

In [58]:
# Agent Workflow Instructions

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [59]:
# Configure Phoenix Tracer
try:
    tracer_provider = register(
    project_name="ollama_phoenix",
    auto_instrument=True,
    endpoint=phoenix_collector_endpoint,
    set_global_tracer_provider=True
)
    display(Markdown("✅ Phoenix tracer registered successfully"))
except Exception as e:
    display(Markdown(f"❌ Phoenix registration failed: {e}"))

Overriding of current TracerProvider is not allowed
DependencyConflict: requested: "openai-agents >= 0.1.0" but found: "openai-agents 0.0.17"
DependencyConflict: requested: "google-genai" but found: "None"


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: ollama_phoenix
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



✅ Phoenix tracer registered successfully

In [60]:
# Ollama Model
ollama_model = OpenAIChatCompletionsModel(
    model=model_llama,
    openai_client=AsyncOpenAI(
        api_key=ollama_api_key, 
        base_url=ollama_base_url
    )
)

In [61]:
# Agents

sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=ollama_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=ollama_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=ollama_model
)

In [62]:
# Phoenix Tracer
async def phoenix_tracer(trace_name, message, *agents):
    tracer = otel_trace.get_tracer(__name__)
    with tracer.start_as_current_span(trace_name) as current_span:
        if agents:
            
            # Trace Attributes
            current_span.set_attribute("user.request", message)
            current_span.set_attribute("model.name", model_llama)
            
            for index, agent in enumerate(agents):
                
                # Trace Attributes
                current_span.set_attribute("agent.name", agent.name)
        
                results = await asyncio.gather(
                    *[Runner.run(agent, message) for agent in agents]
                )

                outputs = []

                for result in results:
                    outputs.append(result.final_output)

                if outputs:
                    total_length = sum(len(output) for output in outputs)
                    
                    # Add response attributes
                    current_span.set_attribute("response.length", total_length)
                    current_span.set_attribute("response.preview", outputs[0][:100] if outputs[0] else "")
        else:
            print("No additional arguments provided.")

        # outputs = [result.final_output for result in results]
        
        
        for output in outputs:
            display(Markdown(output))
        return outputs

In [63]:
result = await phoenix_tracer(
    "Parallel Cold Emails", 
    "Write a cold sales email.", 
    sales_agent1, 
    sales_agent2, 
    sales_agent3
)

Subject: Enhance Your Organization's Compliance with Industry-Grade Security

Dear [Recipient Name],

I noticed that your organization, [Organization Name], has been actively taking steps to maintain the trust of its stakeholders by adhering to industry standards and best practices. However, ensuring constant compliance with SOC 2 requirements can be an ongoing challenge for businesses like yours.

At ComplAI, we understand the complexities involved in preparing for audits and maintaining compliance throughout your organization. That's why our team has developed a comprehensive SaaS tool that utilizes AI to streamline the process of identifying, assessing, and remediating any potential vulnerabilities or areas for improvement.

Our software is designed to guide you through key tasks such as policy development, control implementation, and ongoing assessments to guarantee an efficient and effective SOC 2 process. By leveraging our tool, you can save significant time and resources while ensuring your organization meets the highest industry standards, thus increasing stakeholder confidence.

Would like to schedule a quick call at your earliest convenience to see how ComplAI's solution can address specific compliance challenges your team faces?

To accommodate different time zones or preferences, I'd prefer it if we could fix something up as early as later this week. It will only take about 15 minutes of your time and would give me an opportunity to better understand your requirements.

If you're not the right person for this conversation or cannot participate in a call at such short notice, please feel free to inform me and I'll make sure someone else on my team contacts you instead.

Looking forward to hearing back from you soon.

Here's one:

Subject: Don't Let the Auditors Get the Best of You 

Hey [First Name],

 Congrats! You made it out of last quarter's audit with flying colors (we know, we know, auditor-speak is the worst). But let's be real – you're probably already dreading the next audit. And for good reason: compliance headaches are like having a bad habit you just can't shake.

That's where ComplAI comes in! Our magical SaaS platform uses AI to help your team prep for audits like pros, so you can breathe easy (and save some $$). No more sleepless nights poring over policies or worrying about mislabeled "medium-risks" . 

Ready to trade compliance stress for compliant bliss? We'd love to show you ComplAI's SOC2 powers in action. Let us know when you're free for a quick demo, and we'll whip out the highlights reel.

Cheers,
[Your Name]
@ComplAI
Your compliant sidekicks

P.S: If you're already using [ Competitor X ], you might want to check our free comparison guide – wink-wink.

Here is a sample cold sales email:

Subject: Streamline Your Audit Process with ComplAI

Hi [First Name],

As an IT leader, you know how burdensome compliance audits can be on your team's time and resources. Our experience shows that companies like yours often struggle to maintain accurate documentation and meet SOC2 standards.

That's where ComplAI comes in - our AI-powered tool ensures seamless audit prep, minimizing stress and errors for your teams. With ComplAI, you'll enjoy:

Effortless policy maintenance
Automated control evidence generation
Instant access to audit-ready reports

Curious how ComplAI can transform your compliance journey? Let's schedule a quick demo.

Best,
[Your Name]

P.S. - ComplAI is designed for [SMB/Enterprise] companies like yours. Check out our success stories with other industry leaders on our website.

This email aims to:

* Grab attention with relevance and concision
* Highlight pain points and solution value
* Offer a quick, convenient next step (demo)
* Show credibility with industry focus